In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from itertools import product
import requests
from bs4 import BeautifulSoup
import os

### PRODUCTS EXPIRATION DATA

In [ ]:
# products and their expiration dates
expiration = {
    2: [
        '01. Сэндвич "Классический"', 
        '02. Сэндвич "С курицей"',
        '03. Сэндвич Сырный Соус',
        '04. Сэндвич "PICNIC"',
        '06. Хот - Дог',
        '08. Гамбургер с котлетой',
        '29. Burrito стрипсы'
        ],

    3: [
        '16. Багет пшенично-ржаной',
        '15. Багет пшеничный',
        '21. Тостовый хлеб',
        '23. Батон нарезной',
        '43. Батон к чаю',
        '44. Батон отрубной',
        "45. Батон ''Царский''"
        ],

    5: [
        '10. Булочка Ярославка',
        '59. Булочки для фуда БУРГЕР',
        '60. Булочки для фуда Хот-Дог',
        '18. Лаваш',
        '17. Ролл',
        "42. Батон ''Живая Рожь''"
        ]
}

### ADD NEW DATA TO MAIN DATASET

In [ ]:
def add_new_data_to_initial_dataset():
    # reading data
    initial_DF = pd.read_csv('dataset/initial_dataset.csv')
    print(initial_DF.shape)
    temp_df = pd.read_excel('temp/data.xlsx', skipfooter=3)
    
    # renaming columns
    temp_df.rename({
        'Дата.1': 'Date',
        'Маршрут': 'District',
        'Контрагент': 'Address',
        'Номенклатура': 'Product',
        'Количество': 'Sales',
        'Обмен': 'Returns'
    }, axis=1, inplace=True)
    
    # concating new and old data
    df = pd.concat([initial_DF, temp_df])
    
    # converting a date columns to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # sorting data by date
    df = df.sort_values('Date').reset_index(drop=True).drop_duplicates()
    
    # deleting unnecessary data according to date limit
    date_limit = df['Date'].iloc[-1] - timedelta(weeks=5)
    df = df[df['Date'] >= date_limit]
    
    print(df.shape)
    df.to_csv('dataset/initial_dataset.csv', index=False)

### SHIFTING RETURNS

In [ ]:
def shifting_returns_and_merging_with_sales(initial_DF, days_to_shift):
    if days_to_shift not in [2, 3, 5]:
        return
    
    # reading data
    initial_DF['Date'] = pd.to_datetime(initial_DF['Date'])
    
    # filtering data by products
    df = initial_DF[initial_DF['Product'].isin(expiration[days_to_shift])]
    
    # seperating sales and returns
    df_sales = df.drop(['Returns'], axis=1)
    df_returns = df.drop(['Sales'], axis=1)
    
    # shifting date 
    df_returns['Date'] = df_returns['Date'] - timedelta(days=days_to_shift)
    
    # getting date range of a final df
    date_range = df_sales.merge(
        df_returns,
        on=['Date', 'District', 'Address', 'Product'],
        how='inner'
    )['Date'].unique()
    
    # merging sales and returns
    main_DF = df_sales.merge(
        df_returns,
        on=['Date', 'District', 'Address', 'Product'],
        how='outer'
    )
    
    # final data
    main_DF = main_DF[main_DF['Date'].isin(date_range)].fillna(0)

    return main_DF

In [ ]:
# getting 3 seperate DFs of 3 groups of products with shifted returns
initial_DF = pd.read_csv('dataset/initial_dataset.csv')

two_days_expiricy_DF = shifting_returns_and_merging_with_sales(initial_DF, days_to_shift=2)
three_days_expiricy_DF = shifting_returns_and_merging_with_sales(initial_DF, days_to_shift=3)
five_days_expiricy_DF = shifting_returns_and_merging_with_sales(initial_DF, days_to_shift=5)

### FILL MISSED ADDRESSES

In [ ]:
combined_DF = pd.concat([two_days_expiricy_DF, three_days_expiricy_DF, five_days_expiricy_DF])
combined_DF.sort_values('Date', inplace=True)

In [ ]:
limit_date_to_get_data_about_stores = combined_DF['Date'].iloc[-1] - timedelta(weeks=2)
condition = (combined_DF['Date'] >= limit_date_to_get_data_about_stores)

In [ ]:
all_addresses = combined_DF[condition]['Address'].unique()
all_products = combined_DF[condition]['Product'].unique()

len(all_addresses), len(all_products)

In [ ]:
start_date = combined_DF.iloc[0]['Date']
end_date = combined_DF.iloc[-1]['Date'] + timedelta(weeks=1)

In [ ]:
data = []

# Наполняем список комбиниациями
current_date = start_date
while current_date <= end_date:
    combinations = list(product(all_products, all_addresses))
    data.extend([(current_date, address, product) for product, address in combinations])
    current_date += timedelta(days=1)
    
full_DF = pd.DataFrame(data, columns=["Date", "Address", "Product"])

In [ ]:
# Добавляем колонну продаж и возвратов
full_DF['Sales'] = 0
full_DF['Returns'] = 0

# Далем слияние двух таблиц, чтобы продажи и возвраты встали на место.
main_DF = full_DF[['Date', 'Address', 'Product']].merge(
    combined_DF[['Date', 'Address', 'Product', 'Sales', 'Returns']], 
    on=['Date', 'Address', 'Product'], 
    how='left'
)

# Заполняем 'null' нулями
main_DF = main_DF.fillna(0)

### DELETING ADDRESS-PRODUCT COMBINATIONS THAT ARE NOT RELEVANT

In [ ]:
limit_date_to_get_data_about_store_sales = main_DF.sort_values('Date')['Date'].iloc[-1] - timedelta(weeks=2)
condition = (main_DF['Date'] >= limit_date_to_get_data_about_store_sales)

In [ ]:
grouped_add_prod_sales_DF = main_DF[condition].groupby(['Address', 'Product']).agg({
    'Sales': 'sum'
}).reset_index()

In [ ]:
existing_address_product_combinations_DF = grouped_add_prod_sales_DF[grouped_add_prod_sales_DF['Sales'] > 0].drop('Sales', axis=1)

In [ ]:
existing_address_product_combinations_DF.shape

In [ ]:
main_DF = main_DF.merge(existing_address_product_combinations_DF,
             on=['Address', 'Product'])

### REMOVE SUNDAYS

In [ ]:
main_DF.shape

In [ ]:
main_DF['Weekday'] = main_DF['Date'].dt.weekday
main_DF = main_DF[main_DF['Weekday'] != 6]

In [ ]:
main_DF.shape

### SHIFTING SALES AND RETURNS

In [ ]:
main_DF.sort_values(by=['Address', 'Product', 'Date'], inplace=True)

In [ ]:
days_to_shift = [1, 2, 3, 4, 5, 6, 13, 20]
for day in days_to_shift:
    main_DF[f'Sales_{day + 1}'] = main_DF.groupby(['Address', 'Product'])['Sales'].shift(day)
    main_DF[f'Returns_{day + 1}'] = main_DF.groupby(['Address', 'Product'])['Returns'].shift(day)

In [ ]:
main_DF.fillna(0, inplace=True)

In [ ]:
main_DF.groupby(['Date'])['Sales', 'Sales_2', 'Sales_3', 'Sales_4', 'Sales_5', 'Sales_6', 'Sales_14', 'Sales_21'].sum()

### STATISTICS

In [ ]:
%%time
main_DF['Returns_percentage'] = main_DF['Returns_7'] / (main_DF['Sales_7'] + main_DF['Returns_7'])

main_DF['Mean_sales_in_week'] = main_DF[['Sales_2', 'Sales_3', 'Sales_4', 'Sales_5', 'Sales_6', 'Sales_7']].mean(axis=1)
main_DF['Median_sales_in_week'] = main_DF[['Sales_2', 'Sales_3', 'Sales_4', 'Sales_5', 'Sales_6', 'Sales_7']].median(axis=1)

main_DF['Mean_sales_in_3_weeks'] = main_DF[['Sales_7', 'Sales_14', 'Sales_21']].mean(axis=1)
main_DF['Median_sales_in_3_weeks'] = main_DF[['Sales_7', 'Sales_14', 'Sales_21']].median(axis=1)

main_DF['No_sales_in_week'] = main_DF[['Sales_2', 'Sales_3', 'Sales_4', 'Sales_5', 'Sales_6', 'Sales_7']].sum(axis=1) == 0
main_DF['No_returns_in_week'] = main_DF[['Returns_2', 'Returns_3', 'Returns_4', 'Returns_5', 'Returns_6', 'Returns_7',]].sum(axis=1) == 0

main_DF['Total_sales_in_week'] = main_DF[['Sales_2', 'Sales_3', 'Sales_4', 'Sales_5', 'Sales_6', 'Sales_7']].sum(axis=1)

main_DF['Mean_returns_in_week'] = main_DF[['Returns_2', 'Returns_3', 'Returns_4', 'Returns_5', 'Returns_6', 'Returns_7',]].mean(axis=1)
main_DF['Mean_returns_in_3_weeks'] = main_DF[['Returns_7', 'Sales_14', 'Returns_21']].mean(axis=1)

### SET BACK DISTRICTS

In [ ]:
district_address_DF = combined_DF.sort_values('Date')[['District', 'Address']].drop_duplicates()

In [ ]:
district_address_DF = district_address_DF.drop_duplicates('Address', keep='last')

In [ ]:
district_address_DF['Address'].unique().shape, district_address_DF.shape

In [ ]:
main_DF = main_DF.merge(district_address_DF, on='Address')

In [ ]:
columns_order = list(main_DF.columns)
columns_order.insert(1, columns_order.pop())

In [ ]:
main_DF = main_DF[columns_order]

### HANDLE SPECIAL ORDERS AND ANOMALIES

In [ ]:
def isAnomaly(data):
    # Calculate mean and standard deviation
    mean = np.mean(data)
    std_dev = np.std(data)

    # Calculate quartiles and IQR
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1

    # Calculate bounds for outliers using IQR
    const = 2
    lower_bound = Q1 - const * IQR
    upper_bound = Q3 + const * IQR

    iqr_anomalies = [x for x in data if x < lower_bound or x > upper_bound]
    
    return data['Sales_7'] in iqr_anomalies

In [ ]:
%%time
main_DF['isAnomaly'] = False

condition = main_DF['Sales_7'] > 10
columns_to_check = ['Sales_2', 'Sales_3', 'Sales_4', 'Sales_5', 'Sales_6', 'Sales_7']


main_DF.loc[condition,'isAnomaly'] = main_DF[condition][columns_to_check].apply(isAnomaly, axis=1)

In [ ]:
main_DF['Address'].unique().shape

In [ ]:
main_DF[main_DF['isAnomaly'] == True]['Address'].unique().shape

In [ ]:
main_DF['isAnomaly'].value_counts()

### PREDICT DATAFRAME

In [ ]:
limit_date_to_predict = main_DF.sort_values('Date')['Date'].iloc[-1] - timedelta(days=5)
condition = (main_DF['Date'] == limit_date_to_predict)

predict_DF = main_DF[condition].drop(['Sales', 'Returns'], axis=1)
predict_DF['Predict'] = predict_DF['Sales_7']
predict_DF['Delivered_7'] = predict_DF['Sales_7'] + predict_DF['Returns_7']

predict_DF

### TREND AND WEATHER FUNCTIONS

In [ ]:
def calculate_trend(data):
    y1, y2, y3 = data[0], data[1], data[2]
    trend = ((y3 - y2) + (y2 - y1)) / 2
    
    return trend

In [ ]:
def parse_weather_forecast():
    HEADERS = {
        "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Mobile Safari/537.36",
        "accept": "*/*"
    }

    URL = "https://www.gismeteo.kz/weather-astana-5164/10-days/"
    r = requests.get(URL, headers=HEADERS)
    soup = BeautifulSoup(r.text, 'html.parser')

    date_range = pd.date_range(start=datetime.now().date(), end=(datetime.now() + timedelta(days=9)).date())

    temperature_container = soup.find("div", class_="widget-row-chart widget-row-chart-temperature-air row-with-caption")
    temperature_elements = temperature_container.find_all("temperature-value") # type: ignore
    temperature_values = [temp['value'] for temp in temperature_elements if 'value' in temp.attrs]
    temperature_values = [int(temp) for temp in temperature_values][0::2]

    wind_speed_container = soup.find("div", class_="widget-row widget-row-wind row-wind-gust row-with-caption")
    wind_speed_elements = wind_speed_container.find_all("speed-value") # type: ignore
    wind_speed_values = [temp['value'] for temp in wind_speed_elements if 'value' in temp.attrs]
    wind_speed_values = [int(temp) for temp in wind_speed_values]

    humidity_container = soup.find("div", class_="widget-row widget-row-precipitation-bars row-with-caption")
    humidity_elements = humidity_container.find_all("div", class_= "row-item")

    humidity_values_uncleaned = [item.text for item in humidity_elements]

    humidity_values = []
    for temp in humidity_values_uncleaned:
        value_str = temp.strip().replace(',', '.')

        if value_str:
            humidity_values.append(float(value_str))

    weather_DF = pd.DataFrame({
        'Date': date_range,
        'Temperature': temperature_values,
        'Wind_speed': wind_speed_values,
        'Humidity': humidity_values
    })
    
    return weather_DF

In [ ]:
def parse_weather_past():
    weather_past_DF = pd.read_excel(f"temp/weather/{os.listdir('temp/weather')[0]}", skiprows=6)
    weather_past_DF = weather_past_DF[['Местное время в Астане', 'T', 'Ff', 'U']]

    weather_past_DF.rename({
        'Местное время в Астане': 'Date',
        'T': 'Temperature',
        'Ff': 'wind_speed_values',
        'U': 'humidity_values'
    }, axis=1, inplace=True)

    weather_past_DF['Date'] = pd.to_datetime(weather_past_DF['Date'], dayfirst=True)
    weather_past_DF['Time'] = (weather_past_DF['Date'].dt.time).astype(str)
    
    time_to_filter = ['08:00:00', '11:00:00', '14:00:00', '17:00:00', '20:00:00']
    weather_past_DF = weather_past_DF[weather_past_DF['Time'].isin(time_to_filter)]
    weather_past_DF['Date'] = weather_past_DF['Date'].dt.date
    weather_past_DF = weather_past_DF.drop('Time', axis=1)

    weather_past_DF = weather_past_DF.groupby('Date').mean().reset_index()
    
    return weather_past_DF

In [ ]:
predict_DF['Trend'] = predict_DF[['Sales_21', 'Sales_14', 'Sales_7']].apply(calculate_trend, axis=1)

In [ ]:
weather_past_DF = parse_weather_past()
weather_past_DF['Date'] = pd.to_datetime(weather_past_DF['Date'], dayfirst=True)

In [ ]:
# weather_DF = parse_weather_forecast()
# predict_DF = predict_DF.merge(weather_DF, on='Date', how='left')

### MATH MODEL FORMULA

In [ ]:
predict_DF.fillna(0, inplace=True)

In [ ]:
def categorize_sales(value):
    if value == 0:
        return 'A'
    elif 0 < value <= 10:
        return 'B'
    elif 11 < value <= 20:
        return 'C'
    elif 21 < value <= 30:
        return 'D'
    elif 31 < value:
        return 'E'
    else:
        return None
    
predict_DF['Sales_7_category'] = predict_DF['Sales_7'].apply(categorize_sales)

In [ ]:
def increase_predict(data):
    if data['Sales_7_category'] == 'A':
        return 1
    
    elif data['Sales_7_category'] == 'B':
        return data['Predict'] + 1
    
    elif data['Sales_7_category'] == 'C':
        return data['Predict'] + 2
    
    elif data['Sales_7_category'] == 'D':
        return data['Predict'] * 1.1
    
    elif data['Sales_7_category'] == 'E':
        return data['Predict'] * 1.08
    
    
def decrease_predict(data):
    if data['Sales_7_category'] == 'A':
        return 1

    elif data['Sales_7_category'] == 'B':
        return data['Predict'] - 1
    
    elif data['Sales_7_category'] == 'C':
        return data['Predict'] - 2
    
    elif data['Sales_7_category'] == 'D':
        return data['Predict'] / 1.1
    
    elif data['Sales_7_category'] == 'E':
        return data['Predict'] / 1.08

In [ ]:
predict_DF.shape

In [ ]:
# TREND UP, RETURNS UP, PREDICT UP
condition = (predict_DF['Trend'] > 0) & (predict_DF['Returns_percentage'] > 0.1)
predict_DF.loc[condition, 'Predict'] = predict_DF[condition][['Predict', 'Sales_7_category']].apply(increase_predict, axis=1)

print(predict_DF[condition].shape)

In [ ]:
# TREND DOWN, RETURNS DOWN, PREDICT UP
condition = (predict_DF['Trend'] < 0) & (predict_DF['Returns_percentage'] < 0.08)
predict_DF.loc[condition, 'Predict'] = predict_DF[condition][['Predict', 'Sales_7_category']].apply(increase_predict, axis=1)

print(predict_DF[condition].shape)

In [ ]:
# TREND UP, RETURNS DOWN, PREDICT UP
condition = (predict_DF['Trend'] > 0) & (predict_DF['Returns_percentage'] < 0.08)
predict_DF.loc[condition, 'Predict'] = predict_DF[condition][['Predict', 'Sales_7_category']].apply(increase_predict, axis=1)

print(predict_DF[condition].shape)

In [ ]:
# TREND DOWN, RETURNS UP, PREDICT UP
condition = (predict_DF['Trend'] < 0) & (predict_DF['Returns_percentage'] > 0.1)
predict_DF.loc[condition, 'Predict'] = predict_DF[condition][['Predict', 'Sales_7_category']].apply(decrease_predict, axis=1)

print(predict_DF[condition].shape)

In [ ]:
# NO TREND, RETURNS 8-10%, REMAINS THE SAME
condition = (predict_DF['Trend'] == 0) & (predict_DF['Returns_percentage'] >= 0.08) & (predict_DF['Returns_percentage'] <= 0.1)
predict_DF.loc[condition, 'Predict'] = predict_DF.loc[condition, 'Delivered_7']

predict_DF.loc[condition].shape

In [ ]:
# NO TREND, RETURNS DOWN, PREDICT UP
condition = (predict_DF['Trend'] == 0) & (predict_DF['Returns_percentage'] < 0.08)
predict_DF.loc[condition, 'Predict'] = predict_DF[condition][['Predict', 'Sales_7_category']].apply(increase_predict, axis=1)

predict_DF.loc[condition].shape

In [ ]:
# NO TREND, RETURNS UP, PREDICT DOWN
condition = (predict_DF['Trend'] == 0) & (predict_DF['Returns_percentage'] > 0.1)
predict_DF.loc[condition, 'Predict'] = predict_DF[condition][['Predict', 'Sales_7_category']].apply(decrease_predict, axis=1)

predict_DF.loc[condition].shape

In [ ]:
# TREND UP, RETURNS 8-10%, PREDICT UP
condition = (predict_DF['Trend'] > 0) & (predict_DF['Returns_percentage'] >= 0.08) & (predict_DF['Returns_percentage'] <= 0.1)
predict_DF.loc[condition, 'Predict'] = predict_DF[condition][['Predict', 'Sales_7_category']].apply(increase_predict, axis=1)

predict_DF.loc[condition].shape

In [ ]:
# TREND DOWN, RETURNS 8-10%, PREDICT DOWN
condition = (predict_DF['Trend'] < 0) & (predict_DF['Returns_percentage'] >= 0.08) & (predict_DF['Returns_percentage'] <= 0.1)
predict_DF.loc[condition, 'Predict'] = predict_DF[condition][['Predict', 'Sales_7_category']].apply(decrease_predict, axis=1)

predict_DF.loc[condition].shape

In [ ]:
predict_DF[predict_DF['District'] == '46 маршрут'].to_excel('prediction/46.xlsx', index=False)

In [ ]:
initial_DF['Weekday'] = initial_DF['Date'].dt.weekday + 1
initial_DF[initial_DF['District'] == '01 маршрут'].to_excel('1маршрут.xlsx', index=False)